In [8]:
# define a model using pytorch
# a pretrained model adding layers to classify the 4 classes of the images
import torch.nn as nn
from torchvision import models
from torchvision.models.resnet import ResNet18_Weights

class MIDmodel(nn.Module):
    def __init__(self):
        super(MIDmodel, self).__init__()
        # first pretrained
        # UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
        self.pretrained = models.resnet18(weights=ResNet18_Weights.DEFAULT)
        # add layers
        self.fc1 = nn.Linear(1000, 128)
        self.fc2 = nn.Linear(128, 4)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # set x to three channels
        x = self.pretrained(x)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [9]:
# new model with MobileNetV2
import torch.nn as nn
from torchvision import models
import torch

class MIDMobilemodel(nn.Module):
    def __init__(self):
        super(MIDMobilemodel, self).__init__()
        # create a cnn layer that takes in 3 channels 224x224 and outputs 10 features
        self.cnn1 = nn.Conv2d(3, 10, kernel_size=3, stride=1, padding=1)
        self.cnn2 = nn.Conv2d(10, 30, kernel_size=3, stride=1, padding=1)
        self.cnn3 = nn.Conv2d(30, 50, kernel_size=3, stride=1, padding=1)
        self.cnn4 = nn.Conv2d(50, 30, kernel_size=3, stride=1, padding=1)
        self.cnn5 = nn.Conv2d(30, 10, kernel_size=3, stride=1, padding=1)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(10 * 112 * 112, 4)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.cnn1(x)
        x = self.cnn2(x)
        x = self.cnn3(x)
        x = self.cnn4(x)
        x = self.cnn5(x)
        x = self.maxpool(x)
        x = self.relu(x)
        x = x.reshape(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc1(x)
        return x

In [10]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import pandas as pd
from PIL import Image
import os
class ImageDataset(Dataset):
    def __init__(self, root_dir, patients_ids, patients_df, transform=None):
        # select the rows of the dataframe that correspond to the patients in the list
        self.patients_df = patients_df[patients_df["patient_id"].isin(patients_ids)]
        self.root_dir = root_dir
        self.idxs = set()
        if transform:
            self.transform = transform
            # triplicate the patients_df
            self.patients_df = pd.concat([self.patients_df] * 2, ignore_index=True)
        else:
            self.transform = None
               
            

    def __len__(self):
        return len(self.patients_df)

    def __getitem__(self, idx):
        # get idx row of the dataframe
        row = self.patients_df.iloc[idx]
        # path = root_dir/patientid_examid_spot_framenumber_score.png
        path = os.path.join(self.root_dir, row["patient_id"] + "_" + row["exam_id"] + "_" + row["spot"] + "_" + row["frame_number"] + "_" + row["score"] + ".png")
        image = Image.open(path)
        # if the image is grayscale, convert it to RGB
        if image.mode != "RGB":
            image = image.convert("RGB")

        # Extract label from the image filename
        label = row["score"]
        # normalize the image
        if self.transform:
            if path not in self.idxs:
                self.idxs.add(path)
            else:
                image = self.transform(image)
                self.idxs.remove(path)
        image = transforms.ToTensor()(image)  
        image = transforms.Normalize(mean=[17.29368658, 15.58648964, 15.50668094], std=[32.30078458, 28.69238728, 28.48813082])(image)
        image = transforms.Resize((224, 224))(image)
      
        return image, int(label)

In [11]:
# define dataset
import pandas as pd
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# "patient_id exam_id spot[1-14] frame_number score"
# load patients data in a dataframe from images folder
import glob
images_paths = glob.glob("imagesBW/*.png", recursive=True)
# images are named as: patientid_examid_spotnumber_framenumber_score.png
# create a dataframe with the data removing "images/"
images_df = pd.DataFrame([path[9:-4].split("_") for path in images_paths], columns=["patient_id", "exam_id", "spot", "frame_number", "score"])
images_df["score"] = images_df["score"].astype(str)
images_df["frame_number"] = images_df["frame_number"].astype(str)
images_df["spot"] = images_df["spot"].astype(str)
images_df["patient_id"] = images_df["patient_id"].astype(str)
images_df["exam_id"] = images_df["exam_id"].astype(str)


patients_ids = set(images_df["patient_id"])
patients_ids = list(patients_ids)
# select the top 8 patients by number of frames
# get number of frames for each patient
patients_frames = images_df.groupby("patient_id").count()["score"]
# sort the patients by number of frames
patients_frames = patients_frames.sort_values(ascending=False)
# get the top 8 patients
train_patients = [x for x in patients_frames[:8].index]
# get the rest of the patients
test_patients = [x for x in patients_frames[8:].index]
print("train patients: ", train_patients, "test patients: ", test_patients)

# patients ['1068', '1017', '1045', '1066', '1067', '1052', '1050', '1069', '1047', '1048', '1051']



train patients:  ['1068', '1045', '1048', '1052', '1017', '1067', '1066', '1069'] test patients:  ['1050', '1051', '1047']


In [12]:
# transformations from the paper Deep Learning for Classification and Localization of COVID-19 Markers in Point-of-Care Lung Ultrasound
# were each activated on the image-label pair with a probability of 0.33. The set of augmentation functions, each applied with a randomly sampled strength bounded by a set maximum, consists of:
# - affine transformations (translation (max. ±15%), rotation (max. ±15◦), scaling (max. ±45%), and shearing (max. ±4.5◦))
# - multiplication with a constant (max. ±45%)
# - Gaussian blurring (σmax = 3/4 )
# - contrast distortion (max. ±45%)
# - horizontal flipping (p = 0.5)
# - additive white Gaussian noise (σmax = 0.015)
transformations = transforms.Compose([
    transforms.RandomApply([transforms.RandomAffine(degrees=15, translate=(0.15, 0.15), scale=(0.55, 1.45), shear=4.5)], p=0.5),
    transforms.RandomApply([transforms.ColorJitter(brightness=0.45, contrast=0.45, saturation=0.45, hue=0.45)], p=0.5),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=3, sigma=(0.75))], p=0.5),
    transforms.RandomApply([transforms.RandomHorizontalFlip(p=0.5)], p=0.5)
    ])

In [13]:
import numpy as np
from torch import optim
from tqdm import tqdm

def train_model(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    for inputs, labels in tqdm(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects/ len(dataloader.dataset)
    return epoch_loss, epoch_acc

def test_model(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects / len(dataloader.dataset)
    return epoch_loss, epoch_acc


In [14]:
data_dir = "imagesBW/"
num_epochs = 100
learning_rate = 0.001

model = MIDmodel()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

train_dataset = ImageDataset(data_dir, test_patients, images_df, transformations)
test_dataset = ImageDataset(data_dir, train_patients[:3], images_df)
# define the dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)
results_train = []
results_test = []
# train the model
for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    train_loss, train_acc = train_model(model, train_dataloader, optimizer, criterion, device)
    results_train.append([train_loss, train_acc])
    print(f"Training Loss: {train_loss:.4f} Training Accuracy: {train_acc:.4f}")
    test_loss, test_acc = test_model(model, test_dataloader, criterion, device)
    results_test.append([test_loss, test_acc])
    print(f"Test Loss: {test_loss:.4f} Test Accuracy: {test_acc:.4f}")
    if epoch % 5 == 1:
        torch.save(model.state_dict(), f"MIDmodel_{epoch}.pt")
# print results with commas not dots
print("Train: ", results_train)
print("Test: ", results_test)

# save the model
torch.save(model.state_dict(), f"MIDmodel.pt")



Epoch 0


100%|██████████| 2575/2575 [02:24<00:00, 17.82it/s]


Training Loss: 0.7384 Training Accuracy: 0.7263


100%|██████████| 1276/1276 [02:29<00:00,  8.55it/s]


Test Loss: 2.8759 Test Accuracy: 0.2396
Epoch 1


...

Training Loss: 0.3301 Training Accuracy: 0.8838


100%|██████████| 1276/1276 [02:18<00:00,  9.20it/s]


Test Loss: 3.4717 Test Accuracy: 0.2621
Epoch 3


 62%|██████▏   | 1589/2575 [01:17<00:48, 20.53it/s]


KeyboardInterrupt: 

In [ ]:
# load the model
# model = MIDmodel()
# model.load_state_dict(torch.load("model.pt"))
# model.eval()

In [ ]:
# create a model to predict the score via clustering
class MIDcluser(nn.Module):
    def __init__(self):
        super(MIDcluser, self).__init__()